In [24]:
# import libraries 
import numpy as np
import os
import nibabel as nib

tumours = ['LGG','HGG']

prefix = '/Users/jasonfung/Documents/EECE571' # Jason's Macbook
# prefix = 'C:/Users/Fungj/Documents/EECE_571F' # Jason's Desktop
brats_dir = '/MICCAI_BraTS_2018_Data_Training/'
# prefix = '/home/atom/Documents/datasets/brats' # Adam's Station
file_pattern = '{prefix}/MICCAI_BraTS_2018_Data_Training/{tumour}/{patient_id}/{patient_id}_{contrast}.nii.gz'
# patient_id = 'Brats18_TCIA09_620_1'
contrasts = ['t1ce', 'flair', 't2']
tumours = ['LGG', 'HGG']

data_list_LGG = os.listdir(os.path.join(prefix+brats_dir,tumours[0]))
data_list_HGG = os.listdir(os.path.join(prefix+brats_dir,tumours[1]))
dataset_file_list = data_list_HGG + data_list_LGG

# shuffle and split the dataset file list
import random
random.seed(42)
file_list_shuffled = dataset_file_list.copy()
random.shuffle(file_list_shuffled)
test_ratio = 0.2

train_file, test_file = file_list_shuffled[0:int(len(file_list_shuffled)*(1-test_ratio))], file_list_shuffled[int(len(file_list_shuffled)*(1-test_ratio)):]

if '.DS_Store' in train_file:
    train_file.remove('.DS_Store')


## Calculate Image Statistics

In [26]:
# loop over each raw image and mask to get mean
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
img_dataset_dim = len(train_file)*240*240*155

mean = [0,0,0]
std = [0,0,0]

tmp_sum = [0,0,0]
tmp_sum_sq = [0,0,0]

for name in train_file:
    print("Loading image and masks number: ", name)

    # for HGG tumours
    if name in os.listdir(prefix+brats_dir+'HGG'):
    # load t2 and preprocess the data
        tmp_img_t2 = nib.load(os.path.join(prefix+brats_dir+'HGG'+f'/{name}/{name}_t2.nii.gz')).get_fdata()
        tmp_img_t2 = scaler.fit_transform(tmp_img_t2.reshape(-1, tmp_img_t2.shape[-1])).reshape(tmp_img_t2.shape)
        
        # load t1ce and preprocess the data
        tmp_img_t1ce = nib.load(os.path.join(prefix+brats_dir+'HGG'+f'/{name}/{name}_t1ce.nii.gz')).get_fdata()
        tmp_img_t1ce = scaler.fit_transform(tmp_img_t1ce.reshape(-1, tmp_img_t1ce.shape[-1])).reshape(tmp_img_t1ce.shape)
        
        # load flair and preprocess the data
        tmp_img_flair = nib.load(os.path.join(prefix+brats_dir+'HGG'+f'/{name}/{name}_flair.nii.gz')).get_fdata()
        tmp_img_flair = scaler.fit_transform(tmp_img_flair.reshape(-1, tmp_img_flair.shape[-1])).reshape(tmp_img_flair.shape)
    
    # for LGG tumours
    else:
        tmp_img_t2 = nib.load(os.path.join(prefix+brats_dir+'LGG'+f'/{name}/{name}_t2.nii.gz')).get_fdata()
        tmp_img_t2 = scaler.fit_transform(tmp_img_t2.reshape(-1, tmp_img_t2.shape[-1])).reshape(tmp_img_t2.shape)
        
        # load t1ce and preprocess the data
        tmp_img_t1ce = nib.load(os.path.join(prefix+brats_dir+'LGG'+f'/{name}/{name}_t1ce.nii.gz')).get_fdata()
        tmp_img_t1ce = scaler.fit_transform(tmp_img_t1ce.reshape(-1, tmp_img_t1ce.shape[-1])).reshape(tmp_img_t1ce.shape)
        
        # load flair and preprocess the data
        tmp_img_flair = nib.load(os.path.join(prefix+brats_dir+'LGG'+f'/{name}/{name}_flair.nii.gz')).get_fdata()
        tmp_img_flair = scaler.fit_transform(tmp_img_flair.reshape(-1, tmp_img_flair.shape[-1])).reshape(tmp_img_flair.shape)

    tmp_sum[0] += np.sum(tmp_img_t2)
    tmp_sum[1] += np.sum(tmp_img_t1ce)
    tmp_sum[2] += np.sum(tmp_img_flair)

    tmp_sum_sq[0] += np.sum(tmp_img_t2**2)
    tmp_sum_sq[1] += np.sum(tmp_img_t1ce**2)
    tmp_sum_sq[2] += np.sum(tmp_img_flair**2)


Loading image and masks number:  Brats18_CBICA_ANP_1
Loading image and masks number:  Brats18_TCIA02_608_1
Loading image and masks number:  Brats18_2013_10_1
Loading image and masks number:  Brats18_CBICA_AUQ_1
Loading image and masks number:  Brats18_CBICA_ANG_1
Loading image and masks number:  Brats18_CBICA_AAB_1
Loading image and masks number:  Brats18_TCIA02_118_1
Loading image and masks number:  Brats18_TCIA04_192_1
Loading image and masks number:  Brats18_TCIA12_480_1
Loading image and masks number:  Brats18_CBICA_AQR_1
Loading image and masks number:  Brats18_TCIA02_309_1
Loading image and masks number:  Brats18_TCIA02_607_1
Loading image and masks number:  Brats18_TCIA03_296_1
Loading image and masks number:  Brats18_TCIA02_331_1
Loading image and masks number:  Brats18_TCIA03_133_1
Loading image and masks number:  Brats18_CBICA_AQA_1
Loading image and masks number:  Brats18_TCIA01_390_1
Loading image and masks number:  Brats18_TCIA09_312_1
Loading image and masks number:  Brat

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [34]:
mean = np.array(tmp_sum)/img_dataset_dim
std = np.sqrt(np.array(tmp_sum_sq)/img_dataset_dim - mean**2)

print('Mean normalized pixel: ', mean)
print('Standard deviation pixel: ', std)

Mean normalized pixel:  [0.06250282 0.06072226 0.07404174]
Standard deviation pixel:  [0.15603759 0.15047059 0.18546175]


## Calculate Class Imbalance Weights

In [79]:
def crop_3D(img, new_size):
    img_shape = img.shape
    x_mid = int(img_shape[0]/2)
    y_mid = int(img_shape[1]/2)
    z_mid = int(img_shape[2]/2)

    x_diff = int(abs(new_size[0]-x_mid))
    y_diff = int(abs(new_size[1]-y_mid))
    z_diff = int(abs(new_size[2]-z_mid))

    x_start = x_mid-x_diff
    y_start = y_mid-y_diff
    z_start = z_mid-z_diff

    tmp_img = img[x_start:x_start+new_size[0],y_start:y_start+new_size[1],z_start:z_start+new_size[2]]
    return tmp_img

In [80]:
tot_samples = 0
tot_weights = np.zeros(4)

crop_img = True

for name in train_file:
    # load in the data
    print('Loading in mask of: ', name)
    if name in os.listdir(prefix+brats_dir+'HGG'):
        tmp_mask = nib.load(os.path.join(prefix+brats_dir+'HGG'+f'/{name}/{name}_seg.nii.gz')).get_fdata()
        tmp_mask = tmp_mask.astype(np.uint8)
        tmp_mask[tmp_mask==4] = 3 # reassign value 4 to 3
        if crop_img == True:
            tmp_mask = crop_3D(tmp_mask,(128,128,128))
    else:
        tmp_mask = nib.load(os.path.join(prefix+brats_dir+'LGG'+f'/{name}/{name}_seg.nii.gz')).get_fdata()
        tmp_mask = tmp_mask.astype(np.uint8)
        tmp_mask[tmp_mask==4] = 3 # reassign value 4 to 3
        if crop_img == True:
            tmp_mask = crop_3D(tmp_mask,(128,128,128))
    
    tmp_counts = np.array(np.bincount(tmp_mask.flatten()))

    # some masks dont contain all the labels i.e., label '3' => adding (3,) to (4,) is not possible, so extend the array of counts
    if tmp_counts.shape < tot_weights.shape:
        tmp_counts.resize(tot_weights.shape)

    tot_samples += len(tmp_mask.flatten())
    tot_weights += tmp_counts

class_weights = tot_samples/tot_weights

Loading in mask of:  Brats18_CBICA_ANP_1
hgg
Loading in mask of:  Brats18_TCIA02_608_1
hgg
Loading in mask of:  Brats18_2013_10_1
hgg
Loading in mask of:  Brats18_CBICA_AUQ_1
hgg
Loading in mask of:  Brats18_CBICA_ANG_1
hgg
Loading in mask of:  Brats18_CBICA_AAB_1
hgg
Loading in mask of:  Brats18_TCIA02_118_1
hgg
Loading in mask of:  Brats18_TCIA04_192_1
hgg
Loading in mask of:  Brats18_TCIA12_480_1
lgg
Loading in mask of:  Brats18_CBICA_AQR_1
hgg
Loading in mask of:  Brats18_TCIA02_309_1
hgg
Loading in mask of:  Brats18_TCIA02_607_1
hgg
Loading in mask of:  Brats18_TCIA03_296_1
hgg
Loading in mask of:  Brats18_TCIA02_331_1
hgg
Loading in mask of:  Brats18_TCIA03_133_1
hgg
Loading in mask of:  Brats18_CBICA_AQA_1
hgg
Loading in mask of:  Brats18_TCIA01_390_1
hgg
Loading in mask of:  Brats18_TCIA09_312_1
lgg
Loading in mask of:  Brats18_TCIA10_632_1
lgg
Loading in mask of:  Brats18_2013_16_1
lgg
Loading in mask of:  Brats18_TCIA04_328_1
hgg
Loading in mask of:  Brats18_2013_22_1
hgg
Loa

In [81]:
class_weights

array([  1.01594803, 297.15686617, 106.76578078, 337.13607545])

In [78]:
class_weights


array([  1.01124016, 363.88552568, 157.81788467, 492.44310648])